сэмпл обезличенных и данных для работы, кодировка стандартная - utf-8.

Клиенты - юридические лица и ИП. Тип "списание" обозначает, что транзакция от клиента нашего банка * клиенту другого банка, "зачисление" противоположное значение - от клиента другого банка - клиенту нашего банка *. 

Обозначения в колонках достаточно простые, если будет непонятно - пиши. инн изменены.

Рассматриваем только клиентов нашего банка (для них все считать необходимо):

- в случае транзакции типа "списание" - от кого идут деньги ("from")
- в случае транзакции типа "зачисление" - кому идут деньги ("to")

Датасет неполный. 15 дней, не все компании и транзакции.

Необходимо:

1) разобраться в данных

2) выделить клиентов банка

3) для клиентов банка посчитать характеристики входящих и исходящий финансовых потоков (сумма, количество транзакций, количество компаний-партнеров (с кем транзачит) и так далее).

4) кластеризовать клиетов по выделенным характеристикам

Кластеризация в данном случае необходима, чтобы понять, какие кластеры компаний можно выделить в соответствиии с их транзакционной активностью (мелкий бизнес, крупный бизнес, работа с мелкими договорами и т.д.). Тут надо фантазировать.

In [272]:
import pandas as pd

In [273]:
df = pd.read_csv('data_sample_06_17.csv')

In [274]:
df.tail()

,summa,transaction_type,company_inn_from,company_inn_to,from_type,to_type,from_inn_region,to_inn_region
812563,14050,списание,156655,108384,ЮЛ,ЮЛ,78,77
812564,12899,списание,130864,169245,ЮЛ,ЮЛ,16,77
812565,30000,списание,118189,49191,ЮЛ,ЮЛ,78,74
812566,2000000,списание,178099,92289,ЮЛ,ЮЛ,78,25
812567,3482871,списание,117423,144602,ЮЛ,ЮЛ,77,52


In [275]:
# df.describe()

In [276]:
df.from_type.unique() 

array(['ЮЛ', 'ИП'], dtype=object)

In [277]:
df.transaction_type.unique()        

array(['списание', 'зачисление'], dtype=object)

In [278]:
# df.loc[:, ['transaction_type','company_inn_from']]

In [279]:
df.transaction_type.value_counts()

списание      468524
зачисление    344044
Name: transaction_type, dtype: int64

In [280]:
# group=df.groupby(['transaction_type', 'company_inn_from'])
# group.head()

In [281]:
# df.loc[(df.transaction_type == 'списание')].groupby(['company_inn_from']).summa.agg([len,sum, min, max]).sort_values(by='len', ascending=False)

### Разделяю датасет по типу транзакции, чтобы выиделить наших клиентов

In [282]:
clients_sent = df.loc[(df.transaction_type == 'списание')].rename(columns={'company_inn_from': 'inn'})#.set_index("inn")
clients_sent.head()

,summa,transaction_type,inn,company_inn_to,from_type,to_type,from_inn_region,to_inn_region
0,26150,списание,144897,182381,ЮЛ,ЮЛ,78,77
1,2354,списание,197997,141025,ИП,ЮЛ,78,50
2,449,списание,197997,25474,ИП,ЮЛ,78,50
3,760,списание,197997,25474,ИП,ЮЛ,78,50
4,13714,списание,62379,171735,ИП,ЮЛ,47,77


In [283]:
clients_received = df.loc[(df.transaction_type == 'зачисление')].rename(columns={'company_inn_to': 'inn'})#.set_index("inn")
clients_received.head()

,summa,transaction_type,company_inn_from,inn,from_type,to_type,from_inn_region,to_inn_region
209068,51017,зачисление,195499,35741,ЮЛ,ЮЛ,2,78
209069,58300,зачисление,32291,151272,ЮЛ,ЮЛ,2,78
209070,610,зачисление,156675,94144,ЮЛ,ЮЛ,2,78
209071,34958,зачисление,94947,101867,ЮЛ,ЮЛ,2,78
209072,775,зачисление,164863,94144,ЮЛ,ЮЛ,2,78


#### Feature Engineering

Рейтинг платежа среди всех платежей

In [284]:
clients_sent['send_summ_rank_allpayments'] = clients_sent['summa'].rank(method='first',ascending=False)
clients_sent.sort_values(by = 'send_summ_rank_allpayments')
# clients_sent.shape

clients_received['received_summ_rank_allpayments'] = clients_received['summa'].rank(method='first',ascending=False)
clients_received.sort_values(by = 'received_summ_rank_allpayments')

print(clients_sent.shape,clients_received.shape)

(468524, 9) (344044, 9)


рейтинг клиента по сумме всех его транзакций

In [285]:
tmp = clients_sent.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_sum_rank_inn'})

left = clients_sent.set_index(['inn'])
right = tmp#.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_sum_rank_inn')


tmp = clients_received.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_rank_inn'})

left = clients_received.set_index(['inn'])
right = tmp#.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_sent.sort_values(by = 'received_sum_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 9) (344044, 9)


In [286]:
# result.loc[(result.inn == 149732)].summa.sum()

рейтинг клиента по средней сумме всех его транзакций

In [287]:
tmp = clients_sent.groupby('inn')['summa'].mean().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_sum_mean_rank_inn'})

left = clients_sent#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_sum_mean_rank_inn')

tmp = clients_received.groupby('inn')['summa'].mean().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_mean_rank_inn'})

left = clients_received#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_sent.sort_values(by = 'received_sum_mean_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 10) (344044, 10)


рейтинг клиента по медианной сумме всех его транзакций

In [288]:
tmp = clients_sent.groupby('inn')['summa'].median().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_sum_median_rank_inn'})

left = clients_sent#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_sum_median_rank_inn')

tmp = clients_received.groupby('inn')['summa'].median().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_median_rank_inn'})

left = clients_received#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'received_sum_median_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 11) (344044, 11)


рейтинг клиента по количеству сделанных платежей

In [289]:
tmp = clients_sent.groupby('inn')['summa'].count().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_count_rank_inn'})

left = clients_sent#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_count_rank_inn')

tmp = clients_received.groupby('inn')['summa'].count().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_count_rank_inn'})
left = clients_received#.set_index(['inn'])
right = tmp #.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')
# clients_sent.sort_values(by = 'received_count_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 12) (344044, 12)


рейтинг кому больше всего отправляли наши клиенты по сумме

In [290]:
tmp = clients_sent.groupby('company_inn_to')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'sendto_sum_rank_inn'})

left = clients_sent.reset_index().set_index(['company_inn_to'])#, append=False)
right = tmp #.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'sendto_sum_rank_inn')

tmp = clients_received.groupby('company_inn_from')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_rank_inn'})

left = clients_received.reset_index().set_index(['company_inn_from'])#, append=False)
right = tmp #.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_received.sort_values(by = 'received_sum_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 13) (344044, 13)


рейтинг кому больше всего отправляли наши клиенты по количеству

In [291]:
tmp = clients_sent.groupby('company_inn_to')['summa'].count().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'sendto_count_rank_inn'})

left = clients_sent.reset_index().set_index(['company_inn_to'])#, append=False)
right = tmp #.set_index(['inn'])

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'sendto_count_rank_inn')

tmp = clients_received.groupby('company_inn_from')['summa'].count().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_from_count_rank_inn'})

left = clients_received.reset_index().set_index(['company_inn_from'])#, append=False)
right = tmp #.set_index(['inn'])

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_received.sort_values(by = 'sendto_count_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 14) (344044, 14)


 Рейтинг в какой регион больше всего по сумме отправляли клиенты

In [292]:
tmp = clients_sent.groupby('to_inn_region')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'sendto_region_sum_rank_inn'})

left = clients_sent.reset_index().set_index(['to_inn_region'])
right = tmp 
clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')
# print(clients_sent.shape)
# clients_sent.sort_values(by = 'sendto_region_sum_rank_inn')


tmp = clients_received.groupby('to_inn_region')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'sendto_region_sum_rank_inn'})

left = clients_received.reset_index().set_index(['to_inn_region'])
right = tmp 
clients_received = left.join(right, lsuffix='_base', rsuffix='_join')
# clients_received.sort_values(by = 'sendto_region_sum_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 15) (344044, 15)


 Рейтинг из какого региона больше всего  по сумме отправляли клиенты

In [293]:
tmp = clients_sent.groupby('from_inn_region')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_from_region_sum_rank_inn'})

left = clients_sent.reset_index().set_index(['from_inn_region'])
right = tmp 
clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')
# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_from_region_sum_rank_inn')

tmp = clients_received.groupby('from_inn_region')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_from_region_sum_rank_inn'})

left = clients_received.reset_index().set_index(['from_inn_region'])
right = tmp 
clients_received = left.join(right, lsuffix='_base', rsuffix='_join')
# clients_received.sort_values(by = 'send_from_region_sum_rank_inn')

print(clients_sent.shape,clients_received.shape)

(468524, 16) (344044, 16)


In [294]:
# clients_sent.groupby(['inn','from_inn_region'])['from_inn_region'].nunique().sort_values()

In [295]:
tmp = (clients_sent.groupby(['from_inn_region','inn'])['summa']
        .sum()
        .groupby(level=0)
        .rank(ascending=False, method='dense'))
mi = tmp.index
# type(mi)
tmp = tmp.reset_index()
# tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_from_region_sum_rank_byclient'})
tmp = tmp.drop(columns = ['from_inn_region'])

left = clients_sent.reset_index().set_index(['inn'])
right = tmp.set_index(['inn']) 
clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')
# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_from_region_sum_rank_byclient')


tmp = (clients_received.groupby(['to_inn_region','inn'])['summa']
        .sum()
        .groupby(level=0)
        .rank(ascending=False, method='dense'))
mi = tmp.index
# type(mi)
tmp = tmp.reset_index()
# tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_from_region_sum_rank_byclient'})
tmp = tmp.drop(columns = ['to_inn_region'])

left = clients_received.reset_index().set_index(['inn'])
right = tmp.set_index(['inn']) 
clients_received = left.join(right, lsuffix='_base', rsuffix='_join')
# clients_received.sort_values(by = 'received_from_region_sum_rank_byclient')

print(clients_sent.shape,clients_received.shape)

(468524, 17) (344044, 17)


In [296]:
# tmp = (clients_sent.groupby(['to_inn_region','inn'])['summa']
#         .sum()
#         .groupby(level=0)
#         .rank(ascending=False, method='dense'))
# tmp
# mi = tmp.index
# type(mi)
# tmp = tmp.reset_index()
# # tmp = tmp.to_frame()
# # # tmp.info()
# tmp
# tmp = tmp.rename(columns={'summa':'send_to_region_sum_rank_byclient'})
# tmp.info()
# # tmp = tmp.drop(columns = ['to_inn_region'])
# tmp = tmp.set_index(['inn'])
# tmp
# # # # # # tmp.head(20)
# # # tmp.info()
# # # # # # clients_sent = pd.merge(clients_sent, tmp,on=['inn'])

# # left = clients_sent.reset_index().set_index(['inn'])
# # right = tmp.reset_index(drop=True).set_index(['inn']) 
# # clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')
# # print(clients_sent.shape)

# # clients_sent.sort_values(by = 'send_to_region_sum_rank_byclient')
# # # # .size()

In [297]:
# clients_sent.loc[149732].groupby(['from_inn_region'])['from_inn_region'].nunique().sort_values()

In [298]:
# clients_sent.to_inn_region.value_counts()

рейтинг клиента из региона присутствия банка по сумме всех его платежей

In [299]:
 # не знаю точно все кода
tmp = clients_sent.loc[(clients_sent.from_inn_region.isin([78,77,47,39,54]))]
tmp = tmp.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_sum_home_region_rank'})

left = clients_sent.reset_index().set_index(['inn'])
right = tmp
clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_sum_home_region_rank')

tmp = clients_received.loc[(clients_received.from_inn_region.isin([78,77,47,39,54]))]
tmp = tmp.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_home_region_rank'})

left = clients_received.reset_index().set_index(['inn'])
right = tmp

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_received.sort_values(by = 'send_sum_home_region_rank')

print(clients_sent.shape,clients_received.shape)

(468524, 18) (344044, 18)


рейтинг клиента НЕ из региона присутствия банка по сумме всех его платежей

In [300]:
# region_of_presense = ['78','77','47','39','54']
tmp = clients_sent[clients_sent.from_inn_region.isin([78,77,47,39,54])== False]
tmp = tmp.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'send_sum_another_region_rank'})

left = clients_sent.reset_index().set_index(['inn'])
right = tmp

clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')
# print(clients_sent.shape)
# clients_sent.sort_values(by = 'send_sum_another_region_rank')

tmp = clients_received[clients_received.from_inn_region.isin([78,77,47,39,54])== False]
tmp = tmp.groupby('inn')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
tmp = tmp.to_frame()
tmp = tmp.rename(columns={'summa':'received_sum_another_region_rank'})

left = clients_received.reset_index().set_index(['inn'])
right = tmp

clients_received = left.join(right, lsuffix='_base', rsuffix='_join')

# clients_received.sort_values(by = 'received_sum_another_region_rank')

print(clients_sent.shape,clients_received.shape)

(468524, 19) (344044, 19)


In [301]:
# tmp.loc[(tmp['from_inn_region'] == 77)]#summa.sum()
# clients_sent.loc[(clients_sent.from_inn_region == 77)].sort_values(by = 'send_from_region_sum_rank_byclient')

In [302]:
# clients_sent.loc[9]
# result.loc[(result.inn == 149732)].summa.sum()

 Рейтинг какие юр.лица больше всего  по суммме отправляли

In [303]:
# tmp = clients_sent.groupby('from_type')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
# tmp = tmp.to_frame()
# tmp = tmp.rename(columns={'summa':'send_from_type_sum_rank'})

# left = clients_sent.reset_index().set_index(['from_type'])
# right = tmp

# clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# print(clients_sent.shape)

# clients_sent.sort_values(by = 'send_from_type_sum_rank')

 Рейтинг каким юр.лицам больше всего отправляли по суммме

In [304]:
# tmp = clients_sent.groupby('to_type')['summa'].sum().sort_values(ascending=False).rank(method='dense',ascending=False)
# tmp = tmp.to_frame()
# tmp = tmp.rename(columns={'summa':'send_to_type_sum_rank'})
# tmp
# # clients_sent = pd.merge(clients_sent, tmp,on=['to_type'])
# # clients_sent.head()
# # left = clients_sent.reset_index().set_index(['to_type'])
# # right = tmp

# # clients_sent = left.join(right, lsuffix='_base', rsuffix='_join')

# # print(clients_sent.shape)

# # clients_sent.sort_values(by = 'send_to_type_sum_rank')

In [305]:
# getting Difference

# comparison_column = np.where(clients_sent['send_from_type_sum_rank_base'] == clients_sent['send_from_type_sum_rank_join'],True, False)
# comparison_column
# clients_sent["equal"] = comparison_column
# clients_sent.loc[clients_sent.equal == False]

#### Обьединяем таблицы

In [307]:

# our_clients
left = clients_sent.reset_index().set_index(['inn'])
right = clients_received.reset_index().set_index(['inn'])

our_clients = pd.concat([clients_sent, clients_received])
# our_clients = left.join(right, lsuffix='_sent', rsuffix='_received')
our_clients

,from_inn_region,to_inn_region,company_inn_to,summa,transaction_type,from_type,to_type,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,...,received_summ_rank_allpayments,received_sum_rank_inn_base,received_sum_mean_rank_inn,received_sum_median_rank_inn,received_count_rank_inn,received_sum_rank_inn_join,received_from_count_rank_inn,received_from_region_sum_rank_byclient,received_sum_home_region_rank,received_sum_another_region_rank
inn,,,,,,,,,,,,,,,,,,,,,
9,78,34,189863.0,800000,списание,ЮЛ,ЮЛ,14185.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,78,34,189863.0,2000000,списание,ЮЛ,ЮЛ,5423.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,78,34,189863.0,1000000,списание,ЮЛ,ЮЛ,11499.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,78,34,189863.0,2000000,списание,ЮЛ,ЮЛ,5425.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,78,78,50243.0,413112,списание,ЮЛ,ЮЛ,26592.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201782,78,78,NaN,19900,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,144508.0,9566.0,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN
201782,78,78,NaN,10600,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,182259.0,9566.0,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN
201782,78,78,NaN,11000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,180227.0,9566.0,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN


In [266]:
# my_list = [9]
# our_clients[our_clients.inn.isin(my_list)]


для клиентов банка посчитать характеристики входящих и исходящий финансовых потоков (сумма, количество транзакций, количество компаний-партнеров (с кем транзачит) и так далее).

In [309]:
our_clients_descibe = our_clients.groupby(['inn']).summa.agg([len,sum, min, max]).sort_values(by='len', ascending=False)
our_clients_descibe

,len,sum,min,max
inn,,,,
94144,35869,212840974,0,75699093
185836,18358,1165405618,0,118972908
101867,16655,3072769505,0,69000000
20031,4535,109865490,24,30000000
189385,4164,103580427,151,5566039
...,...,...,...,...
102350,1,1,1,1
105481,1,1097808,1097808,1097808
130354,1,42000,42000,42000


In [ ]:
# clients_sent.loc[(clients_sent.inn == 149732)].summa.sum()

In [ ]:
# our_clients_descibe.len.describe()

In [ ]:
# our_clients.groupby(['inn','transaction_type']).summa.agg([len,sum, min, max]).sort_values(by='len', ascending=False)

In [ ]:
# my_list = [9]
# our_clients[our_clients.index.isin(my_list)]



####   подсчет уникальных контрагентов

In [310]:
our_clients['nuniqe'] = our_clients.groupby(['inn','transaction_type'])['company_inn_to'].transform('nunique')
our_clients.head()

,from_inn_region,to_inn_region,company_inn_to,summa,transaction_type,from_type,to_type,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,...,received_sum_rank_inn_base,received_sum_mean_rank_inn,received_sum_median_rank_inn,received_count_rank_inn,received_sum_rank_inn_join,received_from_count_rank_inn,received_from_region_sum_rank_byclient,received_sum_home_region_rank,received_sum_another_region_rank,nuniqe
inn,,,,,,,,,,,,,,,,,,,,,
9,78,34,189863.0,800000,списание,ЮЛ,ЮЛ,14185.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
9,78,34,189863.0,2000000,списание,ЮЛ,ЮЛ,5423.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
9,78,34,189863.0,1000000,списание,ЮЛ,ЮЛ,11499.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
9,78,34,189863.0,2000000,списание,ЮЛ,ЮЛ,5425.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
9,78,78,50243.0,413112,списание,ЮЛ,ЮЛ,26592.0,1608.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [311]:
our_clients['nuniqe2'] = our_clients.groupby(['inn','transaction_type'])['company_inn_from'].transform('nunique')
our_clients.tail()

,from_inn_region,to_inn_region,company_inn_to,summa,transaction_type,from_type,to_type,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,...,received_sum_mean_rank_inn,received_sum_median_rank_inn,received_count_rank_inn,received_sum_rank_inn_join,received_from_count_rank_inn,received_from_region_sum_rank_byclient,received_sum_home_region_rank,received_sum_another_region_rank,nuniqe,nuniqe2
inn,,,,,,,,,,,,,,,,,,,,,
201782,78,78,NaN,19900,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201782,78,78,NaN,10600,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201782,78,78,NaN,11000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201786,77,78,NaN,50000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,6243.0,4062.0,327.0,9617.0,137.0,7719.0,8779.0,NaN,0,1
201786,77,78,NaN,172000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,6243.0,4062.0,327.0,9617.0,137.0,7719.0,8779.0,NaN,0,1


In [312]:
our_clients = our_clients.rename(columns={'nuniqe': 'q-ty_of_writeoff_client','nuniqe2': 'q-ty_of_senders'})
our_clients.tail()

,from_inn_region,to_inn_region,company_inn_to,summa,transaction_type,from_type,to_type,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,...,received_sum_mean_rank_inn,received_sum_median_rank_inn,received_count_rank_inn,received_sum_rank_inn_join,received_from_count_rank_inn,received_from_region_sum_rank_byclient,received_sum_home_region_rank,received_sum_another_region_rank,q-ty_of_writeoff_client,q-ty_of_senders
inn,,,,,,,,,,,,,,,,,,,,,
201782,78,78,NaN,19900,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201782,78,78,NaN,10600,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201782,78,78,NaN,11000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,13317.0,9503.0,310.0,3798.0,118.0,7632.0,8667.0,NaN,0,2
201786,77,78,NaN,50000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,6243.0,4062.0,327.0,9617.0,137.0,7719.0,8779.0,NaN,0,1
201786,77,78,NaN,172000,зачисление,ЮЛ,ЮЛ,NaN,NaN,NaN,...,6243.0,4062.0,327.0,9617.0,137.0,7719.0,8779.0,NaN,0,1


In [ ]:
# our_clients = our_clients.drop(['q-ty of senders','q-ty of receivers'], axis=1)


In [313]:
our_clients_descibe = our_clients.groupby(['inn','transaction_type','q-ty_of_senders','q-ty_of_writeoff_client']).summa.agg(['count',sum, min, max,'mean','median'])#.sort_values(by='len', ascending=False)
our_clients_descibe

count  \
inn    transaction_type q-ty_of_senders q-ty_of_writeoff_client          
9      зачисление       5               0                            9   
       списание         0               3                            6   
26     зачисление       2               0                            2   
       списание         0               1                            1   
29     списание         0               8                           12   
...                                                                ...   
201782 зачисление       2               0                           19   
       списание         0               6                           13   
201786 зачисление       1               0                            2   
       списание         0               1                            1   
201795 списание         0               8                           15   

                                                                     sum  \
inn    transaction_type q-ty_of_senders q-ty_of_writeoff_client            
9      зачисление       5               0                        5455601   
       списание         0               3                        6218112   
26     зачисление       2               0                          53674   
       списание         0               1                          27200   
29     списание         0               8                         197886   
...                                                                  ...   
201782 зачисление       2               0                         231800   
       списание         0               6                         119946   
201786 зачисление       1               0                         222000   
       списание         0               1                           5500   
201795 списание         0               8                         212198   

                                                                   min  \
inn    transaction_type q-ty_of_senders q-ty_of_writeoff_client          
9      зачисление       5               0                        14578   
       списание         0               3                         5000   
26     зачисление       2               0                         3120   
       списание         0               1                        27200   
29     списание         0               8                         2442   
...                                                                ...   
201782 зачисление       2               0                         9300   
       списание         0               6                            0   
201786 зачисление       1               0                        50000   
       списание         0               1                         5500   
201795 списание         0               8                           59   

                                                                     max  \
inn    transaction_type q-ty_of_senders q-ty_of_writeoff_client            
9      зачисление       5               0                        4515130   
       списание         0               3                        2000000   
26     зачисление       2               0                          50554   
       списание         0               1                          27200   
29     списание         0               8                          59712   
...                                                                  ...   
201782 зачисление       2               0                          31800   
       списание         0               6                          54871   
201786 зачисление       1               0                         172000   
       списание         0               1                           5500   
201795 списание         0               8                          81212   

                                                                         mean  \
inn    transaction_type q-ty_of_senders q-ty_of_writeoff_client                 

4) кластеризовать клиетов по выделенным характеристикам

Кластеризация в данном случае необходима, чтобы понять, какие кластеры компаний можно выделить в соответствиии с их транзакционной активностью (мелкий бизнес, крупный бизнес, работа с мелкими договорами и т.д.). Тут надо фантазировать.

In [329]:
# clients_sent.columns
clients_sent.columns[clients_sent.isna().any()].tolist()

['send_sum_home_region_rank', 'send_sum_another_region_rank']

### Scaling

In [325]:
clients_sent.columns
# left = clients_sent.reset_index().set_index(['inn'])
# right = clients_received.reset_index().set_index(['inn'])

Index(['from_inn_region', 'to_inn_region', 'company_inn_to', 'summa',
       'transaction_type', 'from_type', 'to_type',
       'send_summ_rank_allpayments', 'send_sum_rank_inn',
       'send_sum_mean_rank_inn', 'send_sum_median_rank_inn',
       'send_count_rank_inn', 'sendto_sum_rank_inn', 'sendto_count_rank_inn',
       'sendto_region_sum_rank_inn', 'send_from_region_sum_rank_inn',
       'send_from_region_sum_rank_byclient', 'send_sum_home_region_rank',
       'send_sum_another_region_rank'],
      dtype='object')

In [334]:
import numpy as np

features_scal = ['summa','send_summ_rank_allpayments', 'send_sum_rank_inn',
       'send_sum_mean_rank_inn', 'send_sum_median_rank_inn',
       'send_count_rank_inn', 'sendto_sum_rank_inn', 'sendto_count_rank_inn',
       'sendto_region_sum_rank_inn', 'send_from_region_sum_rank_inn',
       'send_from_region_sum_rank_byclient']

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

# set seed for reproducibility
np.random.seed(0)

original_data = pd.DataFrame(clients_sent[features_scal])
 
                 
scaled_data = minmax_scaling(original_data, columns= features_scal)#['summa'])
scaled_data



,summa,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,send_sum_median_rank_inn,send_count_rank_inn,sendto_sum_rank_inn,sendto_count_rank_inn,sendto_region_sum_rank_inn,send_from_region_sum_rank_inn,send_from_region_sum_rank_byclient
inn,,,,,,,,,,,
9,0.000800,0.030274,0.064927,0.012492,0.010693,0.983871,0.000681,0.136646,0.155556,0.0,0.067760
9,0.002000,0.011573,0.064927,0.012492,0.010693,0.983871,0.000681,0.136646,0.155556,0.0,0.067760
9,0.001000,0.024541,0.064927,0.012492,0.010693,0.983871,0.000681,0.136646,0.155556,0.0,0.067760
9,0.002000,0.011577,0.064927,0.012492,0.010693,0.983871,0.000681,0.136646,0.155556,0.0,0.067760
9,0.000413,0.056755,0.064927,0.012492,0.010693,0.983871,0.024715,0.913043,0.000000,0.0,0.067760
...,...,...,...,...,...,...,...,...,...,...,...
201795,0.000081,0.186535,0.517312,0.618610,0.862702,0.954839,0.000373,0.015528,0.000000,0.0,0.533671
201795,0.000007,0.599578,0.517312,0.618610,0.862702,0.954839,0.000373,0.015528,0.000000,0.0,0.533671
201795,0.000004,0.695276,0.517312,0.618610,0.862702,0.954839,0.000373,0.015528,0.000000,0.0,0.533671


In [336]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=10, random_state=0)
original_data["Cluster_scal"] = kmeans.fit_predict(scaled_data)
original_data.head(2)

,summa,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,send_sum_median_rank_inn,send_count_rank_inn,sendto_sum_rank_inn,sendto_count_rank_inn,sendto_region_sum_rank_inn,send_from_region_sum_rank_inn,send_from_region_sum_rank_byclient,Cluster_scal
inn,,,,,,,,,,,,
9,800000,14185.0,1608.0,315.0,170.0,306.0,32.0,45.0,15.0,1.0,1298.0,4
9,2000000,5423.0,1608.0,315.0,170.0,306.0,32.0,45.0,15.0,1.0,1298.0,4


In [339]:
original_data.Cluster_scal.value_counts()

3    116542
4    115649
0    101434
2     73989
1     60910
Name: Cluster_scal, dtype: int64

#### Standardize

In [341]:
X = clients_sent
# Define a list of the features to be used for the clustering
features = features_scal

features = ['summa','send_summ_rank_allpayments', 'send_sum_rank_inn',
       'send_sum_mean_rank_inn', 'send_sum_median_rank_inn',
       'send_count_rank_inn', 'sendto_sum_rank_inn', 'sendto_count_rank_inn',
       'sendto_region_sum_rank_inn', 'send_from_region_sum_rank_inn',
       'send_from_region_sum_rank_byclient']


# Standardize
original_data = X.loc[:, features]
stdzed_data = (original_data - original_data.mean(axis=0)) / original_data.std(axis=0)
# stdzed_data


In [342]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=10, random_state=0)
original_data["Cluster_stdzed"] = kmeans.fit_predict(stdzed_data)
original_data.head(2)

,summa,send_summ_rank_allpayments,send_sum_rank_inn,send_sum_mean_rank_inn,send_sum_median_rank_inn,send_count_rank_inn,sendto_sum_rank_inn,sendto_count_rank_inn,sendto_region_sum_rank_inn,send_from_region_sum_rank_inn,send_from_region_sum_rank_byclient,Cluster_stdzed
inn,,,,,,,,,,,,
9,800000,14185.0,1608.0,315.0,170.0,306.0,32.0,45.0,15.0,1.0,1298.0,4
9,2000000,5423.0,1608.0,315.0,170.0,306.0,32.0,45.0,15.0,1.0,1298.0,4


In [344]:
original_data.Cluster_stdzed.value_counts()

4    142023
2    125942
0    114458
1     82486
3      3615
Name: Cluster_stdzed, dtype: int64

In [345]:
# original_data.info()

In [331]:
# original_data.columns[original_data.isna().any()].tolist()

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in our_clients.columns if our_clients[cname].nunique() < 10 and 
                        our_clients[cname].dtype == "object"]
low_cardinality_cols

In [ ]:
# # "Cardinality" means the number of unique values in a column
# # Select categorical columns with relatively low cardinality (convenient but arbitrary)
# low_cardinality_cols = [cname for cname in our_clients.columns if our_clients[cname].nunique() < 10 and 
#                         our_clients[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in our_clients.columns if our_clients[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols

# print('Numerical columns: ',numerical_cols)


# print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)

# print(my_cols)

In [ ]:
# our_clients = our_clients.drop(columns=[0,1])
# our_clients

In [ ]:
low_cardinality_cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder

our_clients_encoded3 = our_clients.copy(deep=True)

for col in low_cardinality_cols:

    # creating instance of one-hot-encoder
    enc = OneHotEncoder(handle_unknown='ignore')

    # passing bridge-types-cat column (label encoded values of bridge_types)
    enc_df = pd.DataFrame(enc.fit_transform(our_clients_encoded3[[col]]).toarray())
    # merge with main df bridge_df on key values
    our_clients_encoded3 = our_clients_encoded3.join(enc_df)
    our_clients_encoded3 = our_clients_encoded3.rename(columns={0: f'{col}_0',1: f'{col}_1'})
    our_clients_encoded3 = our_clients_encoded3.drop(columns=[col])
our_clients_encoded3

In [ ]:
# our_clients_encoded3.loc[:,'to_type_1']
our_clients_encoded3.to_type_1.value_counts()

####  колонки с пустыми данными удаляем

In [ ]:
col_nan = our_clients_encoded3.columns[our_clients_encoded3.isna().any()].tolist()
col_nan

In [ ]:
our_clients_encoded_clean = our_clients_encoded3.drop(columns = ['company_inn_to', 'company_inn_from'])
our_clients_encoded_clean

#### Кластеризация по двум признакам

In [ ]:
df_1000 = df.iloc[:10000,:]
X = np.array(list(zip(df_1000['summa'],df_1000['from_inn_region'])))
from sklearn import metrics
silhouette_avgs = []
min_k = 2
#---try k from 2 to maximum number of labels---
for k in range(min_k, 10):
    kmean = KMeans(n_clusters=k).fit(X)
    score = metrics.silhouette_score(X, kmean.labels_)
    print("Silhouette Coefficients for k =", k, "is", score)
    silhouette_avgs.append(score)
#---the optimal k is the one with the highest average silhouette---
Optimal_K = silhouette_avgs.index(max(silhouette_avgs)) + min_k
print("Optimal K is", Optimal_K)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 2
X = np.array(list(zip(df['summa'],df['from_inn_region'])))
kmeans = KMeans(n_clusters=k,n_init=10,max_iter=300,tol=1e-04)
kmeans = kmeans.fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
#---map the labels to colors---
c = ['b','r','y','g','c','m']
colors = [c[i] for i in labels]
plt.scatter(df['summa'],df['from_inn_region'], c=colors, s=2)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', s=100, c='black')
print(centroids)

Мы устанавливаем n_init=10, чтобы 10 раз выполнить кластеризацию K-Means независимо с разными случайными 
центроидами для выбора в качестве финальной модели с наименьшим значением SSE. 
Посредством параметра max_iter мы указываем максимальное количество итераций для каждого запуска (300).
tol - параметра, который управляет порогом изменений во внутрикластерной сумме SSE для объявления сходимости. 
В предыдущем коде мы выбрали порог 1e-04 (= 0.0001). 

In [ ]:
import matplotlib.pyplot as plt

df = our_clients_encoded_clean
plt.scatter(df['from_inn_region'],df['to_inn_region'], c='r', s=2)
plt.xlabel("from_inn_region")
plt.ylabel("to_inn_region")

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 4
X = np.array(list(zip(df['from_inn_region'],df['to_inn_region'])))
kmeans = KMeans(n_clusters=k,n_init=10,max_iter=300,tol=1e-04)
kmeans = kmeans.fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
#---map the labels to colors---
c = ['b','r','y','g','c','m']
colors = [c[i] for i in labels]
plt.scatter(df['from_inn_region'],df['to_inn_region'], c=colors, s=2)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', s=100, c='black')

In [ ]:
df_1000 = df.iloc[:1000,:]
X = np.array(list(zip(df_1000['summa'],df_1000['q-ty_of_writeoff_client'])))
from sklearn import metrics
silhouette_avgs = []
min_k = 2
#---try k from 2 to maximum number of labels---
for k in range(min_k, 10):
    kmean = KMeans(n_clusters=k).fit(X)
    score = metrics.silhouette_score(X, kmean.labels_)
    print("Silhouette Coefficients for k =", k, "is", score)
    silhouette_avgs.append(score)
#---the optimal k is the one with the highest average silhouette---
Optimal_K = silhouette_avgs.index(max(silhouette_avgs)) + min_k
print("Optimal K is", Optimal_K)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 2
X = np.array(list(zip(df['summa'],df['q-ty_of_writeoff_client'])))
kmeans = KMeans(n_clusters=k,init='random',n_init=10,max_iter=300,tol=1e-04)
kmeans = kmeans.fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
#---map the labels to colors---
c = ['b','r','y','g','c','m']
colors = [c[i] for i in labels]
plt.scatter(df['summa'],df['q-ty_of_writeoff_client'], c=colors, s=2)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', s=100, c='black')
print(centroids)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 2
X = np.array(list(zip(df['summa'],df['q-ty_of_writeoff_client'])))
kmeans = KMeans(n_clusters=k,init='k-means++',n_init=10,max_iter=300,tol=1e-04)
kmeans = kmeans.fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
#---map the labels to colors---
c = ['b','r','y','g','c','m']
colors = [c[i] for i in labels]
plt.scatter(df['summa'],df['q-ty_of_writeoff_client'], c=colors, s=2)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', s=100, c='black')
print(centroids)

In [ ]:
import sklearn.cluster as cluster
import seaborn as sns

X = our_clients_encoded_clean

km3 = cluster.KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42).fit(X)
X['Labels']  = km3.labels_
X['Labels'].value_counts()
sns.scatterplot(X['summa'], X['q-ty_of_writeoff_client'], 
hue=labels, palette=sns.color_palette('hls', X['Labels'] .nunique()))
plt.title('KMeans with 3 Clusters')

In [ ]:
# from sklearn import decomposition
# from sklearn import datasets

# X = our_clients_encoded_clean

# pca = decomposition.PCA(n_components=2)
# pca.fit(X)
# X = pca.transform(X)

In [ ]:
# X[:,1]

#### пробую уменьшить размерность и выделить кластеры со всеми числовыми признаками

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

X_train,X_test = train_test_split(X,random_state= 101, train_size = 0.5)

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

X = our_clients_encoded_clean

pca = decomposition.PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)
df = pd.DataFrame(X, columns=['x', 'y'])

kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
fig = plt.figure(figsize=(5, 5))
# colors = map(lambda x: colmap[x+1], labels)
plt.scatter(df['x'], df['y'])#, color=colors) #alpha=0.5, edgecolor='k') 
for idx, centroid in enumerate(centroids):
    plt.scatter(*centroid)#, color=colmap[idx+1]) 
# plt.xlim(0, 80) 
# plt.ylim(0, 80)
plt.show()

#### эксперимент убираем условно категориальные признаки

In [ ]:
our_clients_encoded_clean_reduced = our_clients_encoded_clean.drop(columns= ['inn','from_inn_region','to_inn_region'])

In [ ]:
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

X = our_clients_encoded_clean_reduced

pca = decomposition.PCA(n_components=2)
X = pca.fit_transform(X)
df = pd.DataFrame(X, columns=['x', 'y'])

kmeans = KMeans(n_clusters=6)
kmeans.fit(df)

labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
fig = plt.figure(figsize=(5, 5))
# colors = map(lambda x: colmap[x+1], labels)
plt.scatter(df['x'], df['y'])#, color=colors) #alpha=0.5, edgecolor='k') 
for idx, centroid in enumerate(centroids):
    plt.scatter(*centroid)#, color=colmap[idx+1]) 
# plt.xlim(0, 80) 
# plt.ylim(0, 80)
plt.show()



In [ ]:
print(labels)
print(centroids)

In [ ]:
#---map the labels to colors---
c = ['b','r','y','g','c','m']
colors = [c[i] for i in labels]
plt.scatter(df['x'],df['y'], c=colors, s=18)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', s=100, c='black')

In [ ]:
# df['x'][0,0]
df.iloc[0, 0]

In [ ]:
from sklearn.decomposition import PCA
import time

X = our_clients_encoded_clean_reduced
print(f'До PCA содержит: {X.shape[1]}признаков')

pca = PCA(n_components=2)
tik = time.time()
X_pca = pca.fit_transform(X)


print(f'После PCA содержит: {X_pca.shape[1]}признаков')
print(f'Дисперсия составляет % от оригинала:  {sum(pca.explained_variance_ratio_)}')

print(f'PCA выполнялся с постоянным потреблением памяти сек: {time.time()- tik}')




# df = pd.DataFrame(X, columns=['x', 'y'])

# kmeans = KMeans(n_clusters=3)
# kmeans.fit(X)

# labels = kmeans.predict(X)
# centroids = kmeans.cluster_centers_
# fig = plt.figure(figsize=(5, 5))
# colors = map(lambda x: colmap[x+1], labels)

plt.scatter(X_pca[:,0], X_pca[:,1])#, color=colors) #alpha=0.5, edgecolor='k')
# for idx, centroid in enumerate(centroids):
#     plt.scatter(*centroid)#, color=colmap[idx+1]) 
# # plt.xlim(0, 80) 
# # plt.ylim(0, 80)
plt.show()

#### оценка точности если есть у

In [ ]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# def test_classification_accuracy(X_in,y_in):
#     X_train,X_test,y_train,y_test = \
#     train_test_split(X_in,y_in,random_state= 101, train_size = 0.5)
    
#     clf = SGDClassifier('log',random_state= 101)
#     clf.fit(X_train,y_train)
#     return accuracy_score(y_test,clf.predict(X_test))


# print(f'Точность SGDClassifier до PCA: {test_classification_accuracy(X,y)}')
# print(f'Точность после PCA: {test_classification_accuracy(X_pca,y)}')

#### IncrementalPCA (без размещения всего набора данных в памяти)

In [ ]:
from sklearn.decomposition import IncrementalPCA
import time

X = our_clients_encoded_clean_reduced

pca = IncrementalPCA(None,batch_size=1000)

tik = time.time()
for i in range(100):
    pca.partial_fit(X)
print(f'IncrementalPCA выполнялся с постоянным потреблением памяти сек: {time.time()- tik}')

#### MiniBatchSparsePCA (хорош,когда количество признаков больше количества наблюдений). Выполнялся очень долго.

In [ ]:
from sklearn.decomposition import MiniBatchSparsePCA
import time

X = our_clients_encoded_clean_reduced

tik = time.time()
pca = MiniBatchSparsePCA(20,method='cd',random_state=101,n_iter=1000)
pca.fit(X)


print(f'Разряженный алгоритм PCA выполнялся с постоянным потреблением памяти сек: {time.time()- tik}')

In [ ]:
import matplotlib as mpl
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

X = our_clients_encoded_clean_reduced

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

mpl.rcParams['figure.figsize'] = (10,5)


for K in [2,3,4]:
    cls = KMeans(n_clusters = K,random_state=101)
    y_pred = cls.fit_predict(X_pca)
    plt.subplot(1,3,K-1)
    plt.title(f'K средних: {K}')
    plt.scatter(X_pca[:,0],X_pca[:,1],c=y_pred,edgecolors = 'none')
    plt.scatter(cls.cluster_centers_[:,0],cls.cluster_centers_[:,1],\
               marker = 'x',color = 'r',s=100,linewidth=4)
plt.show()
mpl.rcParams['figure.figsize'] = (5,5)

#### Определение оптимального количества кластеров. Метод локтя

In [ ]:
# X=X_pca
X=our_clients_encoded_clean_reduced.iloc[:10000,:]

Ks =  range(2,10)
Ds = []
for K in Ks:
    cls= KMeans(n_clusters=K,random_state=101)
    cls.fit(X)
    Ds.append(cls.inertia_)
plt.plot(Ks,Ds,'o-')
plt.xlabel('Величина К')
plt.ylabel('Искажение')
plt.show()

#### Силуэтная отметка ( указывает, что все данные  находятся в наилучшем кластере). слишком долгая

In [ ]:
from sklearn.metrics import silhouette_score
X=our_clients_encoded_clean_reduced.iloc[:10000,:]

Ks =  range(2,10)
Ds = []
for K in Ks:
    cls= KMeans(n_clusters=K,init='k-means++', max_iter=300, n_init=10,random_state=101)
    Ds.append(silhouette_score(X, cls.fit_predict(X)))
    
plt.plot(Ks,Ds,'o-')
plt.xlabel('Величина К')
plt.ylabel('Силуэтная отметка')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from matplotlib import cm
from sklearn.metrics import silhouette_samples 

X=our_clients_encoded_clean_reduced.iloc[:10000,:]

km = KMeans(n_clusters = 2,init='k-means++', max_iter=300, n_init=10,random_state=101) 
y_km = km.fit_predict(X)


cluster_labels = np.unique(y_km) 
n_clusters = cluster_labels.shape[0] 
silhouette_vals = silhouette_samples (X,y_km,metric='euclidean') 
y_ax_lower, y_ax_upper = 0, 0
yticks = []
for i, с in enumerate(cluster_labels): 
    c_silhouette_vals = silhouette_vals [y_km == с] 
    c_silhouette_vals.sort() 
    y_ax_upper += len(c_silhouette_vals) 
    color = cm.jet(float(i) / n_clusters) 
    plt.barh(range(y_ax_lower, y_ax_upper),c_silhouette_vals,height=1.0,color=color) #,edgecolor='none',
    yticks.append((y_ax_lower + y_ax_upper) / 2.) 
    y_ax_lower += len(c_silhouette_vals)
silhouette_avg = np.mean(silhouette_vals) 
plt.axvline(silhouette_avg,color="red", linestyle="--") 
plt.yticks(yticks, cluster_labels + 1) 
plt.ylabel('Кластер') 
plt.xlabel('Коэффициент силуэта') 
plt.tight_layout()
plt.figure(figsize=(10,20))
plt.show()

In [ ]:
X=our_clients_encoded_clean_reduced.iloc[:10000,:]

from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 2, affinity='euclidean', linkage='complete')
labels = ac.fit_predict(X)
print('Метки кластеров: %s' % labels) 


In [ ]:
from sklearn. cluster import DBSCAN 

X = our_clients_encoded_clean_reduced
pca = PCA(n_components=2)
X = pca.fit_transform(X)

db = DBSCAN(eps=0.2, min_samples=5, metric='euclidean') 
у_db = db.fit_predict(X)
plt.scatter(X[у_db == 0, 0], X[у_db == 0, 1], c = 'lightblue', edgecolor='black', marker='o',s=40, label = 'кластер 1') 
plt.scatter(X[у_db == 1, 0], X[у_db == 1, 1], c = 'red', edgecolor='black', marker='s' , s=40, label = 'кластер 2') 
plt. legend () 

plt. tight_layout () 
plt. show ()

In [ ]:
# X = df
# from sklearn import metrics
# silhouette_samples = metrics.silhouette_samples(X, kmeans.labels_)
# print(silhouette_samples)
# print("Average of Silhouette Coefficients for k =", k)
# print("============================================")
# print("Silhouette mean:", silhouette_samples.mean())

In [ ]:
# X = df
# print("Silhouette mean:", metrics.silhouette_score(X, kmeans.labels_))

In [ ]:
# X = our_clients_encoded_clean_reduced
# pca = PCA(n_components=2)
# X = pca.fit_transform(X)


# silhouette_avgs = []
# min_k = 2

# #---try k from 2 to maximum number of labels---
# for k in range(min_k, len(X)):
#     kmean = KMeans(n_clusters=k).fit(X)
#     score = metrics.silhouette_score(X, kmean.labels_)
#     print("Silhouette Coefficients for k =", k, "is", score)
#     silhouette_avgs.append(score)
# f, ax = plt.subplots(figsize=(7, 5))
# ax.plot(range(min_k, len(X)), silhouette_avgs)
# plt.xlabel("Number of clusters")
# plt.ylabel("Silhouette Coefficients")
# #---the optimal k is the one with the highest average silhouette---
# Optimal_K = silhouette_avgs.index(max(silhouette_avgs)) + min_k
# print("Optimal K is ", Optimal_K)

#### несколько тестов на размер датасета и количество кластеров

In [ ]:
# piece_of_data = 
time_results = {4:[],8:[],12:[]}
set_sizes = [20000,200000,500000]

for set_size in set_sizes:
    print(f'Размер набора данных: {set_size}')
    X = X[:set_size,:]
    
    for K in [4,8,12]:
        print(f'K: {K}')
        cls = KMeans(K,random_state = 101)
        timeit = %timeit -o -n1 -r1 cls.fit(X)
        time_results[K].append(timeit.best)
        
plt.plot(set_sizes,time_results[4],'r',label='K=4')
plt.plot(set_sizes,time_results[8],'g',label='K=8')
plt.plot(set_sizes,time_results[12],'b',label='K=12')

plt.xlabel('Размер тренировочного набора')
plt.ylabel('Время тренировки')
plt.legend(loc=0)
plt.show()

In [ ]:
import time
from sklearn.cluster import MiniBatchKMeans

cls = MiniBatchKMeans(12,batch_size=1000,random_state=101)
ts = []

tik = time.time()
for i in range(100):
    cls.partial_fit(X)
    ts.append(time.time()-tik)

plt.plot(range(len(ts)),ts)
plt.xlabel('Тренировочные пакеты')
plt.ylabel('Время,сек')
plt.show()

In [ ]:
X=X_pca

Ks =  list(range(4,10))
Ds = []
           
for K in Ks:
    print(f'K: {K}')
    cls= MiniBatchKMeans(K,batch_size = 1000,random_state=101)
    for i in range(100):
        cls.partial_fit(X)
    Ds.append(cls.inertia_)
           
plt.plot(Ks,Ds,'o-')
plt.xlabel('Величина К')
plt.ylabel('Искажение')
plt.show()

In [ ]:
# from sklearn.metrics import homogeneity_completeness_v_measure

# X=X_pca

# Ks =  range(2,10)
# HCVs = []

# for K in Ks:
#     y_pred = KMeans(n_clusters=K,random_state=101).fit_predict(X)
#     HCVs.append(homogeneity_completeness_v_measure(y_pred))
    
# plt.plot(Ks,[el[2] for el in HCVs],'b--',label = 'V-мера')
# # plt.xlabel('Величина К')
# # plt.ylabel('Искажение')
# plt.show()

In [ ]:
# df = pd.DataFrame(X, columns=['x', 'y'])
# df

In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure(1, figsize=(4, 3))
# plt.clf()
# ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

# plt.cla()

# for name, label in [('Setosa', 0), ('Versicolour', 1), ('Virginica', 2)]:
#     ax.text3D(X[0].mean(),
#               X[1].mean() + 1.5,
#               X[2].mean(), name,
#               horizontalalignment='center',
#               bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))
# # Reorder the labels to have colors matching the cluster results
# y = np.choose(y, [1, 2, 0]).astype(float)
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap=plt.cm.nipy_spectral,
#            edgecolor='k')

# ax.w_xaxis.set_ticklabels([])
# ax.w_yaxis.set_ticklabels([])
# ax.w_zaxis.set_ticklabels([])

# plt.show()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, low_cardinality_cols)
    ])


model = KMeans(n_clusters=3)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
# Preprocessing of training data, fit model 
my_pipeline.fit(our_clients)

In [ ]:
labels = my_pipeline.predict(our_clients)
centroids = model.cluster_centers_

In [ ]:
our_clients.head(2)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# fig = plt.figure(figsize=(5, 5))
# colors = map(lambda x: colmap[x+1], labels)
# plt.scatter(our_clients['summa'], our_clients['q-ty of receivers'], color=colors, alpha=0.5, edgecolor='k') 
# for idx, centroid in enumerate(centroids):
#     plt.scatter(*centroid, color=colmap[idx+1]) 
# plt.xlim(0, 80)
# plt.ylim(0, 80)
# plt.show()

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# onehotencoder = OneHotEncoder(categorical_features = [0])
# x = onehotencoder.fit_transform(our_clients).toarray()

In [ ]:
our_clients = our_clients.drop(columns=[0,1])
our_clients

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)